In [22]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import optuna
from datetime import datetime
from preprocess.data_loader import load_data
from preprocess.preprocessing import preprocess
from preprocess.feature_selection import select_features
from models.dnn import build_dnn
from models.traditional import train_traditional_models
from evaluations.evaluation import evaluate_model
from configs import config
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from pytorch_tabnet.tab_model import TabNetClassifier
import os

In [23]:
# ---------- (1) 각 모델별 objective 함수 정의 ----------

def objective_dnn(trial, X_train, y_train):
    # 아키텍처(은닉층 구성)와 드롭아웃 비율 탐색
    layers = trial.suggest_categorical(
        'layers',
        [(512, 256), (1024, 512, 256), (1024, 512, 256, 128)]
    )
    dropout = trial.suggest_float('dropout', 0.1, 0.5)

    model = build_dnn(X_train.shape[1], layers, dropout=dropout, optimizer='Adam')
    history = model.fit(
        X_train, y_train,
        epochs=30, batch_size=32,
        validation_split=0.2, verbose=0
    )
    return max(history.history['val_accuracy'])

def objective_rf(trial, X_train, y_train):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 5, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10)
    }
    rf = RandomForestClassifier(**params, random_state=config.SEED)
    return cross_val_score(rf, X_train, y_train, cv=3, scoring='accuracy').mean()

def objective_xgb(trial, X_train, y_train):
    import xgboost as xgb
    from sklearn.model_selection import cross_val_score
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 400),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0)
    }
    clf = xgb.XGBClassifier(**params, random_state=config.SEED, use_label_encoder=False, eval_metric='mlogloss')
    return cross_val_score(clf, X_train, y_train, cv=3, scoring='accuracy').mean()

def objective_lgbm(trial, X_train, y_train):
    import lightgbm as lgb
    from sklearn.model_selection import cross_val_score
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 400),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0)
    }
    clf = lgb.LGBMClassifier(**params, random_state=config.SEED)
    return cross_val_score(clf, X_train, y_train, cv=3, scoring='accuracy').mean()

def objective_svm(trial, X_train, y_train):
    from sklearn.svm import SVC
    from sklearn.model_selection import cross_val_score
    params = {
        'C': trial.suggest_float('C', 0.1, 10.0, log=True),
        'kernel': trial.suggest_categorical('kernel', ['rbf', 'linear', 'poly']),
        'gamma': trial.suggest_categorical('gamma', ['scale', 'auto'])
    }
    clf = SVC(**params, probability=True, random_state=config.SEED)
    return cross_val_score(clf, X_train, y_train, cv=3, scoring='accuracy').mean()

def objective_lr(trial, X_train, y_train):
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import cross_val_score
    params = {
        'C': trial.suggest_float('C', 0.01, 10.0, log=True),
        'solver': trial.suggest_categorical('solver', ['lbfgs', 'liblinear'])
    }
    clf = LogisticRegression(**params, max_iter=500, random_state=config.SEED)
    return cross_val_score(clf, X_train, y_train, cv=3, scoring='accuracy').mean()

def objective_tabnet(trial, X_train, y_train, X_valid, y_valid):
    params = {
        'n_d': trial.suggest_int('n_d', 8, 64),
        'n_a': trial.suggest_int('n_a', 8, 64),
        'n_steps': trial.suggest_int('n_steps', 3, 10),
        'gamma': trial.suggest_float('gamma', 1.0, 2.0),
        'lambda_sparse': trial.suggest_float('lambda_sparse', 1e-5, 1e-1, log=True)
    }
    clf = TabNetClassifier(**params, seed=config.SEED)
    clf.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        patience=10, max_epochs=100, batch_size=1024
    )
    preds = clf.predict(X_valid)
    return evaluate_model(y_valid, preds)['accuracy']


In [24]:
# ---------- (2) 메인실행 ----------

def main(search_time_minutes=5):
    # 데이터 로딩 및 전처리
    df_original = load_data(config.DATA_PATH)
    df, numeric_cols, categorical_cols = preprocess(df_original)
    df['mh_PHQ_S_grouped'] = df['mh_PHQ_S'].apply(lambda x: 0 if x <= 4 else 1 if x <= 9 else 2)
    X = df.drop(['mh_PHQ_S', 'mh_PHQ_S_grouped'], axis=1)
    y = df['mh_PHQ_S_grouped']
    X = pd.DataFrame(SimpleImputer(strategy='mean').fit_transform(X), columns=X.columns)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=config.SEED)
    X_train_selected, selected_features, selector = select_features(X_train, y_train, config.SELECTED_FEATURES)
    X_test_selected = selector.transform(X_test)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_selected)
    X_test_scaled = scaler.transform(X_test_selected)
    
    # 탐색 시간(초) 설정
    timeout = search_time_minutes * 1

    # ------ (3) 모델 리스트에 원하는 objective 추가 ------
    model_objectives = [
        ('DNN', objective_dnn),
        ('RandomForest', objective_rf),
        ('XGBoost', objective_xgb),
        ('LightGBM', objective_lgbm),
        ('SVM', objective_svm),
        ('LogisticRegression', objective_lr),
    ]
    studies = {}
    for model_name, objective in model_objectives:
        print(f"\n[Optuna] {model_name} 하이퍼파라미터 튜닝 시작")
        study = optuna.create_study(direction='maximize')
        study.optimize(lambda trial: objective(trial, X_train_scaled, y_train), timeout=timeout)
        studies[model_name] = study
        print(f"[Optuna] {model_name} Best score: {study.best_value}, Best params: {study.best_params}")
            # TabNet만 별도로(입력 float32)
    print("\n[Optuna] TabNet 하이퍼파라미터 튜닝 시작")
    tabnet_study = optuna.create_study(direction='maximize')
    tabnet_study.optimize(
        lambda trial: objective_tabnet(
            trial,
            X_train_scaled.astype(np.float32), y_train.values,
            X_test_scaled.astype(np.float32), y_test.values
        ),
        timeout=timeout
    )
    studies['TabNet'] = tabnet_study
    print(f"[Optuna] TabNet Best score: {tabnet_study.best_value}, Best params: {tabnet_study.best_params}")
   # ------ (4) Best 파라미터로 성능 재학습/측정 ------
    model_records = []
    param_records = []

    for model_name, study in studies.items():
        best_params = study.best_params
        print(f"\n[최종 모델 학습] {model_name} | best_params: {best_params}")
        if model_name == 'DNN':
            model = build_dnn(
                X_train_scaled.shape[1],
                best_params['layers'],
                dropout=best_params['dropout']
            )
            model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)
            y_pred = model.predict(X_test_scaled).argmax(axis=1)
        elif model_name == 'RandomForest':
            from sklearn.ensemble import RandomForestClassifier
            model = RandomForestClassifier(**best_params, random_state=config.SEED)
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
        elif model_name == 'XGBoost':
            import xgboost as xgb
            model = xgb.XGBClassifier(**best_params, random_state=config.SEED, use_label_encoder=False, eval_metric='mlogloss')
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
        elif model_name == 'LightGBM':
            import lightgbm as lgb
            model = lgb.LGBMClassifier(**best_params, random_state=config.SEED)
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
        elif model_name == 'SVM':
            from sklearn.svm import SVC
            model = SVC(**best_params, probability=True, random_state=config.SEED)
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
        elif model_name == 'LogisticRegression':
            from sklearn.linear_model import LogisticRegression
            model = LogisticRegression(**best_params, max_iter=500, random_state=config.SEED)
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
        elif model_name == 'TabNet':
            model = TabNetClassifier(**best_params, seed=config.SEED)
            model.fit(X_train_scaled.astype(np.float32), y_train.values, max_epochs=100, batch_size=1024)
            y_pred = model.predict(X_test_scaled.astype(np.float32))
        else:
            continue

        metrics = evaluate_model(y_test, y_pred)
        model_records.append({'Model': model_name, **metrics})
        param_records.append({'Model': model_name, 'Hyperparameters': best_params})

    # ------ (5) 결과 저장 ------
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    os.makedirs("reports", exist_ok=True)
    excel_filename = f"reports/{now}_detailed_model_report.xlsx"
    with pd.ExcelWriter(excel_filename, engine='xlsxwriter') as writer:
        pd.DataFrame(model_records).to_excel(writer, sheet_name='Performance', index=False)
        pd.DataFrame(param_records).to_excel(writer, sheet_name='Hyperparameters', index=False)
    print(f"\n모든 모델 결과가 '{excel_filename}'에 저장되었습니다.")

if __name__ == "__main__":
    main(search_time_minutes=5)

[I 2025-06-17 00:22:49,119] A new study created in memory with name: no-name-a544fd32-57d7-43b1-8453-8bb905760394



[Optuna] DNN 하이퍼파라미터 튜닝 시작


/root/myenv/lib/python3.12/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (512, 256) which is of type tuple.
  warnings.warn(message)
/root/myenv/lib/python3.12/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (1024, 512, 256) which is of type tuple.
  warnings.warn(message)
/root/myenv/lib/python3.12/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (1024, 512, 256, 128) which is of type tuple.
  warnings.warn(message)
/root/myenv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential 

[Optuna] DNN Best score: 0.8480446934700012, Best params: {'layers': (1024, 512, 256, 128), 'dropout': 0.45974702987733895}

[Optuna] RandomForest 하이퍼파라미터 튜닝 시작


[I 2025-06-17 00:23:05,212] Trial 0 finished with value: 0.8504024895082652 and parameters: {'n_estimators': 250, 'max_depth': 15, 'min_samples_split': 9}. Best is trial 0 with value: 0.8504024895082652.
[I 2025-06-17 00:23:07,358] Trial 1 finished with value: 0.8448123491133228 and parameters: {'n_estimators': 409, 'max_depth': 28, 'min_samples_split': 2}. Best is trial 0 with value: 0.8504024895082652.
[I 2025-06-17 00:23:08,404] Trial 2 finished with value: 0.8495080850501968 and parameters: {'n_estimators': 216, 'max_depth': 27, 'min_samples_split': 6}. Best is trial 0 with value: 0.8504024895082652.
[I 2025-06-17 00:23:10,209] Trial 3 finished with value: 0.8459304672179235 and parameters: {'n_estimators': 361, 'max_depth': 29, 'min_samples_split': 4}. Best is trial 0 with value: 0.8504024895082652.
[I 2025-06-17 00:23:10,210] A new study created in memory with name: no-name-ee1dda42-41ef-4e8c-a9f4-8509ad9de28c
/root/myenv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarn

...

[Optuna] TabNet Best score: 0.839142091152815, Best params: {'n_d': 31, 'n_a': 59, 'n_steps': 3, 'gamma': 1.2808957449155602, 'lambda_sparse': 0.000319414706685981}

[최종 모델 학습] DNN | best_params: {'layers': (1024, 512, 256, 128), 'dropout': 0.45974702987733895}
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step

[최종 모델 학습] RandomForest | best_params: {'n_estimators': 250, 'max_depth': 15, 'min_samples_split': 9}

[최종 모델 학습] XGBoost | best_params: {'n_estimators': 155, 'max_depth': 8, 'learning_rate': 0.1774275966648856, 'subsample': 0.8145226031687541}


/root/myenv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [00:24:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



[최종 모델 학습] LightGBM | best_params: {'n_estimators': 111, 'max_depth': 10, 'learning_rate': 0.018785241035251175, 'subsample': 0.9426541494630125}
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000102 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 383
[LightGBM] [Info] Number of data points in the train set: 4472, number of used features: 11
[LightGBM] [Info] Start training from score -0.402228
[LightGBM] [Info] Start training from score -2.229724
[LightGBM] [Info] Start training from score -1.497836

[최종 모델 학습] SVM | best_params: {'C': 0.6844619858225305, 'kernel': 'linear', 'gamma': 'auto'}

[최종 모델 학습] LogisticRegression | best_params: {'C': 0.10587757114207608, 'solver': 'liblinear'}

[최종 모델 학습] TabNet | best_params: {'n_d': 31, 'n_a': 59, 'n_steps': 3, 'gamma': 1.2808957449155602, 'lambda_sparse': 0.000319414706685981}
epoch 0  | 

/root/myenv/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/root/myenv/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 1  | loss: 0.75721 |  0:00:00s
epoch 2  | loss: 0.62052 |  0:00:00s
epoch 3  | loss: 0.58739 |  0:00:00s
epoch 4  | loss: 0.55685 |  0:00:00s
epoch 5  | loss: 0.50375 |  0:00:00s
epoch 6  | loss: 0.50461 |  0:00:00s
epoch 7  | loss: 0.4871  |  0:00:00s
epoch 8  | loss: 0.48564 |  0:00:00s
epoch 9  | loss: 0.47916 |  0:00:00s
epoch 10 | loss: 0.4714  |  0:00:01s
epoch 11 | loss: 0.47023 |  0:00:01s
epoch 12 | loss: 0.46859 |  0:00:01s
epoch 13 | loss: 0.47329 |  0:00:01s
epoch 14 | loss: 0.46771 |  0:00:01s
epoch 15 | loss: 0.46487 |  0:00:01s
epoch 16 | loss: 0.46419 |  0:00:01s
epoch 17 | loss: 0.45148 |  0:00:01s
epoch 18 | loss: 0.46222 |  0:00:01s
epoch 19 | loss: 0.45709 |  0:00:01s
epoch 20 | loss: 0.46398 |  0:00:02s
epoch 21 | loss: 0.44753 |  0:00:02s
epoch 22 | loss: 0.45005 |  0:00:02s
epoch 23 | loss: 0.45791 |  0:00:02s
epoch 24 | loss: 0.45281 |  0:00:02s
epoch 25 | loss: 0.45521 |  0:00:02s
epoch 26 | loss: 0.45792 |  0:00:02s
epoch 27 | loss: 0.45294 |  0:00:02s
e